## Example Usage of CwmsDataStore and DssDataStore Classes ##

### Setup ###

In [1]:
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))

import json
import platform
import time
from datetime import datetime
from typing import Optional, cast

from hec import CwmsDataStore, DeleteAction, DssDataStore, HecTime, Location, TimeSeries

with open(os.path.join(os.path.dirname(os.getcwd()), ".vscode", "launch.json")) as f:
    config = json.load(f)
for key, value in config["configurations"][0].get("env", {}).items():
    if key.startswith("cda_api_"):
        os.environ[key] = value

### Catalog/Retrieve CWMS Time Series and Store to HEC-DSS File ###

In [2]:
dss_file_name = (
    "T:/test_dss.dss" if platform.system() == "Windows" else "/tmp/test_dss.dss"
)


# NOTE: The following environment variables are used if set:
#   cda_api_root
#   cda_api_key
#   cda_api_office
with CwmsDataStore.open() as db:
    # can also use:
    #   db = CwmsDataStore.open(<api_url>[, api_key=<api_key])
    #   db.office = "SWT"
    end_time = HecTime.now().label_as_time_zone("local")
    start_time = cast(HecTime, (end_time - "P1M"))

    # --------------------------------- #
    # catalog locations in the database #
    # --------------------------------- #
    print("\nCataloging databse locations with kind of PROJECT")
    catalog = db.catalog(
        "location", kind="PROJECT", fields="name,latitude,longitude,elevation,unit"
    )
    print("\n\tProject\tlatitude\tlongitude\televation")
    for catalog_item in sorted(catalog):
        name, lat, lon, elev, unit = catalog_item.split("\t")
        print(
            f"\t{name}\t{float(lat):-8.4f}\t{float(lon):-8.4f}\t{float(elev):7.2f} {unit}"
        )
    # ------------------------------------- #
    # retrieve a location from the database #
    # ------------------------------------- #
    loc = cast(Location, db.retrieve("keys"))
    print("\nRetrieving location KEYS")
    print(f"\toffice              = {loc.office}")
    print(f"\tname                = {loc.name}")
    print(f"\tkind                = {loc.kind}")
    print(f"\ttime_zone           = {loc.time_zone}")
    print(f"\tlatitude            = {loc.latitude}")
    print(f"\tlongitude           = {loc.longitude}")
    print(f"\thorizontal_datum    = {loc.horizontal_datum}")
    print(f"\televation           = {loc.elevation}")
    print(f"\tvertical_datum      = {loc.vertical_datum}")
    print(f"\tvertical_datum_info = {loc.vertical_datum_json}")
    # ----------------------------------- #
    # catalog time series in the database #
    # ----------------------------------- #
    print(f"\nCataloging time series in database matching keys*elev*ccp-rev")
    catalog = db.catalog(
        "timeseries", pattern="keys*elev*ccp-rev", fields="name,latest-time"
    )
    db.time_window = (
        start_time.convert_to_time_zone("UTC"),
        end_time.convert_to_time_zone("UTC"),
    )
    latest_times = {}
    print("\n\tLatest Time\t\tTime Series ID")
    for catalog_item in catalog:
        tsid, latest_time_str = catalog_item.split("\t")
        print(f"\t{latest_time_str}\t{tsid}")
        latest_times[tsid] = HecTime(latest_time_str).convert_to_time_zone("local")
    # ----------------------------------------------- #
    # create a new location and store to the database #
    # ----------------------------------------------- #
    new_loc = Location(
        name=f"TestLoc_{int(datetime.now().timestamp())}",
        office=loc.office,
        kind="SITE",
        time_zone=loc.time_zone,
        latitude=cast(float, loc.latitude) + 1.0,
        longitude=cast(float, loc.longitude) - 1.0,
        horizontal_datum=loc.horizontal_datum,
        elevation=1000.0,
        elevation_unit="ft",
        vertical_datum="NGVD29",
    )
    print(f"\nStoring new location {new_loc}")
    db.is_read_only = False
    db.store(new_loc)
    # ------------------------------------------- #
    # retrieve the new location from the database #
    # ------------------------------------------- #
    loc2 = cast(Location, db.retrieve(new_loc.name))
    print(f"\nRetrieving location {new_loc.name}")
    print(f"\toffice              = {loc2.office}")
    print(f"\tname                = {loc2.name}")
    print(f"\tkind                = {loc2.kind}")
    print(f"\ttime_zone           = {loc2.time_zone}")
    print(f"\tlatitude            = {loc2.latitude}")
    print(f"\tlongitude           = {loc2.longitude}")
    print(f"\thorizontal_datum    = {loc2.horizontal_datum}")
    print(f"\televation           = {loc2.elevation}")
    print(f"\tvertical_datum      = {loc2.vertical_datum}")
    print(f"\tvertical_datum_info = {loc2.vertical_datum_json}")
    with DssDataStore.open(dss_file_name, read_only=False) as dss:
        # can also use
        #   dss = DssDataStore.open(<filename>)
        #   dss.is_read_only = False

        # --------------------------------- #
        # copy time series from CWMS to DSS #
        # --------------------------------- #
        print("\nRetrieving time series from CWMS and storing to DSS")
        for tsid in latest_times:
            if latest_times[tsid] >= start_time:
                ts = cast(TimeSeries, db.retrieve(tsid)).convert_to_time_zone("local")
                print(f"\t{ts}")
                ts.version = f"CWMS-{ts.version}"
                dss.store(ts)
            else:
                print(f"\t-->No data in time window for {tsid}")
        # --------------------------- #
        # catalog the DSS time series #
        # --------------------------- #
        print("\nDSS catalog")
        catalog = dss.catalog(
            "timeseries", pattern="//keys/elev*/cwms-ccp-rev/", condensed=True
        )
        for dataset_name in catalog:
            print(f"\t{dataset_name}")
        # --------------------------------- #
        # copy time series from DSS to CWMS #
        # --------------------------------- #
        print("\nRetrieving time series from DSS and storing to CWMS")
        for dataset_name in catalog:
            ts = cast(TimeSeries, dss.retrieve(dataset_name))
            ts.iset_location(new_loc)
            print(f"\t{ts}")
            db.store(ts)
        # ----------------------------------- #
        # catalog new time series in database #
        # ----------------------------------- #
        print("\nCataloging new time series in database")
        catalog = db.catalog("timeseries", pattern=f"{new_loc.name}.*")
        for tsid in catalog:
            print(f"\t{tsid}")
        # ------------------------------------ #
        # delete new time series from database #
        # ------------------------------------ #
        print("\nDeleting new time series from database")
        for tsid in catalog:
            db.delete(tsid, delete_action=DeleteAction.DELETE_ALL.name)
        print("\nCataloging new time series")
        catalog = db.catalog("timeseries", pattern=f"{new_loc.name}.*")
        print(f"\tnumber of time seires matching {new_loc.name}* = {len(catalog)}")
    # ----------------------------------------- #
    # delete the new location from the database #
    # ----------------------------------------- #
    time.sleep(1.0)
    print(f"\nDeleting new location {new_loc}")
    try:
        db.delete(new_loc.name, delete_action=DeleteAction.DELETE_ALL.name)
    except:
        print("-->Error during delete location!")
    # ---------------------------------------- #
    # catalog the new location in the database #
    # ---------------------------------------- #
    print(f"\nCataloging location {new_loc.name}")
    catalog = db.catalog("location", pattern=new_loc.name)
    print(f"\tnumber of locations matching {new_loc.name} = {len(catalog)}")


Cataloging databse locations with kind of PROJECT


ERROR:root:CDA Error: response=<Response [500]>



	Project	latitude	longitude	elevation
	ALTU	 34.8875	-99.2964	1500.00 ft
	ARBU	 34.4444	-97.0300	 800.00 ft
	ARCA	 35.6483	-97.3631	 965.00 ft
	BIGH	 37.2700	-95.4700	 600.00 ft
	BIRC	 36.5347	-96.1625	 690.00 ft
	BROK	 34.1431	-94.6944	 500.00 ft
	CANT	 36.0842	-98.6014	1575.00 ft
	CHEN	 37.7267	-97.7933	1400.00 ft
	CHOU	 35.8572	-95.3705	 452.50 ft
	COPA	 36.8856	-95.9714	   0.00 ft
	COUN	 38.6792	-96.5069	1225.00 ft
	DENI	 33.8181	-96.5722	   0.00 ft
	ELDR	 37.8464	-96.8222	1271.50 ft
	ELKC	 37.2775	-95.7769	 742.00 ft
	EUFA	 35.3069	-95.3625	 498.00 ft
	FALL	 37.6467	-96.0775	 902.50 ft
	FCOB	 35.1650	-98.4617	1279.00 ft
	FGIB	 35.8711	-95.2286	 483.00 ft
	FOSS	 35.5383	-99.1783	1563.00 ft
	FSUP	 36.5539	-99.5711	1974.50 ft
	GSAL	 36.7444	-98.1356	1100.50 ft
	HEYB	 35.9478	-96.2986	 718.00 ft
	HUDS	 36.2301	-95.1822	 555.00 ft
	HUGO	 34.0117	-95.3803	 351.50 ft
	HULA	 36.9289	-96.0883	 685.50 ft
	JOHN	 38.2367	-95.7683	 995.00 ft
	KAWL	 36.6994	-96.9217	1000.00 ft
	KEMP	 33.7584	-